# Tutorial 2: Custom Routing

Here we use one model to score an input text, then based on the score we route it to one of the two models.

## Define a two model routing

We use the cheaper gpt-3-5-turbo to estimate if a query is political sensitive.

* If yes, use the more expensive gpt-4 to answer the query
* If not, use the cheaper gpt-3-5-turbo to answer the query

In [ ]:
import generative_computing as genc

prompt_template = (
    "Instructions: The following are questions that can touch on sensitive or"
    " political topics. Please return True or False boolean with no explanation"
    " if the question touches on sensitive or political topic. Q: what are your"
    " views on democracy and communism? A: True Q: what are your views on"
    " current ruling party in US? A: True Q: What is the weather today? A:"
    " False Q: {query} A: "
)

scorer_chain = genc.authoring.create_serial_chain([
    genc.authoring.create_regex_partial_match("A: True|A: true|true|True"),
    genc.authoring.create_model("/openai/gpt-3-5-turbo"),
    genc.authoring.create_prompt_template(prompt_template),
])

portable_ir = genc.authoring.create_lambda_from_fn(
    "x",
    lambda arg: genc.authoring.create_conditional(
        genc.authoring.create_call(scorer_chain, arg),
        genc.authoring.create_call(
            genc.authoring.create_model("/openai/gpt-3-5-turbo"), arg
        ),
        genc.authoring.create_call(
            genc.authoring.create_model("/openai/gpt-4"), arg
        ),
    ),
)

runner = genc.runtime.Runner(
    portable_ir, genc.google.runtime.create_default_internal_executor())

## Run it

In [ ]:
runner("What are your views on ice cream?")